In [1]:
from collections import Counter
import math
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

In [2]:
corpus_of_doc=["High school students are in a crucial stage for career exploration", "needing guidance to make informed decisions for their future.", "This qualitative research analyzes the factors influencing their career choices through interviews, observations, and documentation.", "The findings reveal that intelligence and special talents, career interests, personality, and family background and socioeconomic status significantly impact students' decisions.", "Internal factors like personal abilities and interests", "along with external factors such as family influence and socio-economic conditions", "play a critical role in shaping their career paths.", "Understanding these factors can help educators and counselors provide better career guidance to students."]

In [3]:
corpus_of_doc

['High school students are in a crucial stage for career exploration',
 'needing guidance to make informed decisions for their future.',
 'This qualitative research analyzes the factors influencing their career choices through interviews, observations, and documentation.',
 "The findings reveal that intelligence and special talents, career interests, personality, and family background and socioeconomic status significantly impact students' decisions.",
 'Internal factors like personal abilities and interests',
 'along with external factors such as family influence and socio-economic conditions',
 'play a critical role in shaping their career paths.',
 'Understanding these factors can help educators and counselors provide better career guidance to students.']

In [4]:
user_query="I am an Indian and I live in India"

In [5]:
document="India is a country for the Indians and for everyone"

## Chunking


In [6]:
#Chunks of query
query_tokens=user_query.lower().split(" ")

In [7]:
#Chunks of ducument
document_tokens=document.lower().split(" ")

In [8]:
document_tokens

['india',
 'is',
 'a',
 'country',
 'for',
 'the',
 'indians',
 'and',
 'for',
 'everyone']

## Embedding and Similarity Search

[link text](https://)

In [9]:
#Frequery of word using for Embedding for Query
query_counter=Counter(query_tokens)

In [10]:
query_counter

Counter({'i': 2,
         'am': 1,
         'an': 1,
         'indian': 1,
         'and': 1,
         'live': 1,
         'in': 1,
         'india': 1})

In [11]:
#Frequery of word using for Embedding for Document
document_counter=Counter(document_tokens)

In [12]:
document_counter

Counter({'for': 2,
         'india': 1,
         'is': 1,
         'a': 1,
         'country': 1,
         'the': 1,
         'indians': 1,
         'and': 1,
         'everyone': 1})

In [13]:
#Sentence Vectors(NOt very good result with sentence vector)
lst=[]
for token in query_counter.keys():
  lst.append(query_counter[token])

In [14]:
lst

[2, 1, 1, 1, 1, 1, 1, 1]

In [15]:
for tokens in query_counter.keys() & document_counter.keys():
  print(tokens)

india
and


In [16]:
#Dot product for Similarity Search
#Words Embedding
mylist=[]
for token in query_counter.keys() & document_counter.keys():
  mylist.append(query_counter[token]*document_counter[token])


In [17]:
query_magnitude=math.sqrt(sum(query_counter[token]**2 for token in query_counter))

In [18]:
mylist

[1, 1]

In [19]:
dot_product=sum(mylist)

In [20]:
query_magnitude


3.3166247903554

In [21]:
document_magnitude=math.sqrt(sum(document_counter[token]**2 for token in document_counter))

In [22]:
document_magnitude

3.4641016151377544

In [23]:
#Formula for Cosine similarity search
similarity=(dot_product)/(query_magnitude*document_magnitude)

In [24]:
similarity

0.17407765595569785

In [25]:
def cosine_similarity2(query,document):
  #Tokenize and convert to lowercase
  query_tokens = query.lower().split(" ")
  document_tokens = document.lower().split(" ")

  # Create Counters for query and document
  query_counter = Counter(query_tokens)
  document_counter = Counter(document_tokens)

  # Calculate dot product
  dot_product = sum(query_counter[token] * document_counter[token] for token in query_counter.keys() & document_counter.keys())

  # Calculate magnitudes
  query_magnitude = math.sqrt(sum(query_counter[token] ** 2 for token in query_counter))
  document_magnitude = math.sqrt(sum(document_counter[token] ** 2 for token in document_counter))

  # Calculate cosine similarity
  similarity = dot_product / (query_magnitude * document_magnitude) if query_magnitude * document_magnitude != 0 else 0

  return similarity

In [26]:
cosine_similarity2(user_query,document)

0.17407765595569785

In [27]:
def return_response(query,corpus):
  similarities=[]
  for doc in corpus:
    similarity=cosine_similarity2(query,doc)
    similarities.append(similarity)

  return corpus_of_doc[similarities.index(max(similarities))]


In [28]:
user_query="I am a high School Student"


In [29]:
relevent_document=return_response(user_query,corpus_of_doc)

In [30]:
# augument this response by using llama3 model


In [31]:
import requests
import json
full_response = []

In [32]:
full_response = []
prompt = """
You are a bot that makes recommendations for activities. You answer in very short sentences and do not include extra information.
This is the recommended activity: {relevant_document}
The user input is: {user_input}
Compile a recommendation to the user based on the recommended activity and the user input.
"""

url = 'http://localhost:11434/api/generate'


data = {
    "model": "llama3",
    "prompt": prompt.format(user_input=user_query, relevant_document=relevent_document)
}

headers = {'Content-Type': 'application/json'}

response = requests.post(url, data=json.dumps(data), headers=headers, stream=True)


try:
    for line in response.iter_lines():
  # filter out keep-alive new lines
      if line:
        decoded_line = json.loads(line.decode('utf-8'))
        #print(decoded_line['response'])  # uncomment to results, token by token
        full_response.append(decoded_line['response'])
finally:
  response.close()

print(''.join(full_response))

Explore career options through internships or mentorship programs.
